In [278]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from mlb.Data import Data

In [279]:
df = pd.read_csv('data/games.csv')

In [280]:
df

,Game,away,away-record,awayaway-record,home,home-record,homehome-record,away-score,home-score,postseason info,...,LOSS - Pitcher - Id,LOSS - Pitcher - Name,LOSS - Pitcher - AbbrName,LOSS - Pitcher - Record,SAVE - Pitcher - Stats,SAVE - Pitcher - Id,SAVE - Pitcher - Name,SAVE - Pitcher - AbbrName,SAVE - Pitcher - Record,Extra Innings
0,360403123,STL,0-1,0-1 Away,PIT,1-0,1-0 Home,1.0,4.0,NaN,...,5403.0,Adam Wainwright,A. Wainwright,(0-1),NaN,NaN,NaN,NaN,NaN,NaN
1,360403130,TOR,1-0,1-0 Away,TB,0-1,0-1 Home,5.0,3.0,NaN,...,31003.0,Chris Archer,C. Archer,(0-1),"1.0 IP, 0 ER, 2 K, 0 BB",32693.0,Roberto Osuna,R. Osuna,(1),NaN
2,360403107,NYM,0-1,0-1 Away,KC,1-0,1-0 Home,3.0,4.0,NaN,...,31214.0,Matt Harvey,M. Harvey,(0-1),"1.0 IP, 0 ER, 2 K, 1 BB",28957.0,Wade Davis,W. Davis,(1),NaN
3,360404108,SF,1-0,1-0 Away,MIL,0-1,0-1 Home,12.0,3.0,NaN,...,31140.0,Wily Peralta,W. Peralta,(0-1),NaN,NaN,NaN,NaN,NaN,NaN
4,360404101,MIN,0-1,0-1 Away,BAL,1-0,1-0 Home,2.0,3.0,NaN,...,29265.0,Kevin Jepsen,K. Jepsen,(0-1),NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13434,401361977,ATL,88-73,46-35 Away,HOU,95-67,51-30 Home,2.0,7.0,"World Series - Game 2, ATL Wins series 4-2",...,32685.0,Max Fried,M. Fried,(0-1),NaN,NaN,NaN,NaN,NaN,NaN
13435,401361978,HOU,95-67,44-37 Away,ATL,88-73,42-38 Home,0.0,2.0,"World Series - Game 3, ATL Wins series 4-2",...,4684365.0,Luis Garcia,L. Garcia,(0-1),"1.0 IP, 0 ER, 0 K, 0 BB",31549.0,Will Smith,W. Smith,(1),NaN
13436,401361979,HOU,95-67,44-37 Away,ATL,88-73,42-38 Home,2.0,3.0,"World Series - Game 4, ATL Wins series 4-2",...,41261.0,Cristian Javier,C. Javier,(0-1),"1.0 IP, 0 ER, 1 K, 0 BB",31549.0,Will Smith,W. Smith,(2),NaN
13437,401361980,HOU,95-67,44-37 Away,ATL,88-73,42-38 Home,9.0,5.0,"World Series - Game 5, ATL Wins series 4-2",...,36133.0,A.J. Minter,A.J. Minter,(1-1),NaN,NaN,NaN,NaN,NaN,NaN


In [281]:
df.columns

Index(['Game', 'away', 'away-record', 'awayaway-record', 'home', 'home-record',
       'homehome-record', 'away-score', 'home-score', 'postseason info',
       'Walks Issued - Away', 'Walks Issued - Home', 'Stolen Bases - Away',
       'Stolen Bases - Home', 'Strikeouts Thrown - Away',
       'Strikeouts Thrown - Home', 'Total Bases - Away', 'Total Bases - Home',
       'Stadium', 'Date', 'Location', 'Odds', 'O/U', 'Attendance', 'Capacity',
       'Duration', 'Umpires', 'WIN - Pitcher - Stats', 'WIN - Pitcher - Id',
       'WIN - Pitcher - Name', 'WIN - Pitcher - AbbrName',
       'WIN - Pitcher - Record', 'LOSS - Pitcher - Stats',
       'LOSS - Pitcher - Id', 'LOSS - Pitcher - Name',
       'LOSS - Pitcher - AbbrName', 'LOSS - Pitcher - Record',
       'SAVE - Pitcher - Stats', 'SAVE - Pitcher - Id',
       'SAVE - Pitcher - Name', 'SAVE - Pitcher - AbbrName',
       'SAVE - Pitcher - Record', 'Extra Innings'],
      dtype='object')

In [282]:
def normalize_columns(cols):
    new_cols = []
    for col in cols.copy():
        replace_dict = {' - ': '_',
                       '-': '_',
                       ' ': '_',
                       '/': '_'}
        temp = col.lower()
        for k, v in replace_dict.items():
            temp = temp.replace(k, v)

        new_cols.append(temp)
    return new_cols

In [283]:
df.columns = normalize_columns(df.columns)
df.columns

Index(['game', 'away', 'away_record', 'awayaway_record', 'home', 'home_record',
       'homehome_record', 'away_score', 'home_score', 'postseason_info',
       'walks_issued_away', 'walks_issued_home', 'stolen_bases_away',
       'stolen_bases_home', 'strikeouts_thrown_away', 'strikeouts_thrown_home',
       'total_bases_away', 'total_bases_home', 'stadium', 'date', 'location',
       'odds', 'o_u', 'attendance', 'capacity', 'duration', 'umpires',
       'win_pitcher_stats', 'win_pitcher_id', 'win_pitcher_name',
       'win_pitcher_abbrname', 'win_pitcher_record', 'loss_pitcher_stats',
       'loss_pitcher_id', 'loss_pitcher_name', 'loss_pitcher_abbrname',
       'loss_pitcher_record', 'save_pitcher_stats', 'save_pitcher_id',
       'save_pitcher_name', 'save_pitcher_abbrname', 'save_pitcher_record',
       'extra_innings'],
      dtype='object')

In [284]:
df.dtypes

game                        int64
away                       object
away_record                object
awayaway_record            object
home                       object
home_record                object
homehome_record            object
away_score                float64
home_score                float64
postseason_info            object
walks_issued_away         float64
walks_issued_home         float64
stolen_bases_away         float64
stolen_bases_home         float64
strikeouts_thrown_away    float64
strikeouts_thrown_home    float64
total_bases_away          float64
total_bases_home          float64
stadium                    object
date                       object
location                   object
odds                       object
o_u                        object
attendance                 object
capacity                   object
duration                   object
umpires                    object
win_pitcher_stats          object
win_pitcher_id            float64
win_pitcher_na

In [285]:
def clean_games(df):
    df = df.set_index('game')
    df['datetime'] = pd.to_datetime(df['date'])
    df['date'] = df.datetime.dt.date
    df['time'] = df.datetime.dt.time
    df = df.sort_values('datetime')
    df = pd.concat([df[['date', 'home', 'away']],
                    df.drop(['date', 'home', 'away'], axis=1)], axis=1)
    df = df.drop(df[df.duplicated()].index)
    df = df.replace('(null-null)', np.nan)
    for col in df.columns:
        if df[col].isna().sum() < 100:
            df = df.drop(df[df[col].isna()].index)
        if col == 'save_pitcher_record':
            df[col] = df[col].fillna('(0)')

        if 'record' in col:
            if 'pitcher' in col:
                df[col] = df[col].str.replace(r'\((\d+-?\d*)\)', r'\1', regex=True)

                if 'save' in col:
                    df[col.replace('record', 'saves')] = df[col].str.replace(r'\((\d+)\)', r'\1', regex=True).astype(int)
                    df = df.drop(col, axis=1)
                    continue

            df[col.replace('record', 'wins')] = df[col].str.replace(r'(\d+)-{1}(\d+)\s*\w*', r'\1', regex=True).astype(int)
            df[col.replace('record', 'losses')] = df[col].str.replace(r'(\d+)-{1}(\d+)\s*\w*', r'\2', regex=True).astype(int)
            df = df.drop(col, axis=1)
        if 'postseason' in col:
            df['regular_season'] = df[col].isna()
            df[col] = df[col].fillna('Regular Season')

        if col == 'stadium':
            df[col] = df[col].str.replace(r'^\s*(.+(\s\w+)*){1}(.*\s*)*', r'\1', regex=True)
        if col == 'location':
            df[col] = df[col].str.replace(r'^(.+(\s\w+)*)\s*\d*',r'\1', regex=True)
        if col in ['attendance', 'capacity']:
            df[col] = df[col].str.replace(',','')
            df[col] = pd.to_numeric(df[col])
            df[col] = df[col].fillna(int(df[col].mean()))
        if col == 'duration':
            df['hours'] = df[col].str.split(':').apply(lambda x: int(x[0]))
            df['minutes'] = df[col].str.split(':').apply(lambda x: int(x[1][:2]))
            df[col] = df['minutes'] + 60 * df['hours']
            df = df.drop(['minutes', 'hours'], axis=1)

        if 'stats' in col:
            if 'save' in col:
                df[col] = df[col].fillna('0.0 IP, 0 ER, 0 K, 0 BB')
            stat_regex = r'(\d)\.(\d) IP, (\d+) ER, (\d+) K, (\d+) BB'
            df[col.replace('stats','full')] = df[col].str.replace(stat_regex, r'\1', regex=True).apply(lambda x: int(x))
            df[col.replace('stats', 'partial')] = df[col].str.replace(stat_regex, r'\2', regex=True).apply(lambda x: int(x))
            df[col.replace('stats', 'outs')]  = df[col.replace('stats','full')] * 3 + df[col.replace('stats','partial')]
            df = df.drop([col.replace('stats','full'), col.replace('stats', 'partial')], axis=1)
            df[col.replace('stats', 'er')] = df[col].str.replace(stat_regex, r'\3', regex=True).apply(lambda x: int(x))
            df[col.replace('stats', 'k')] = df[col].str.replace(stat_regex, r'\4', regex=True).apply(lambda x: int(x))
            df[col.replace('stats', 'bb')] = df[col].str.replace(stat_regex, r'\5', regex=True).apply(lambda x: int(x))
            df = df.drop(col, axis=1)
        if 'extra' in col:
            df[col] = df[col].fillna(False)
        if col == 'odds':
            temp = df[col].str.extract(r'Line:\s(\w+)(?:\s-(\d+))?')
            temp.columns = ['fav','odds']
            def proces_row(row):
                if row['fav'] == 'EVEN':
                    row['odds'] = '100'
                return row
            temp = temp.apply(proces_row, axis=1)
            temp.odds = temp.odds.astype(float)
            df[col] = temp.odds
            df['fav'] = temp.fav
        if col == 'o_u':
            df[col] = df[col].str.extract(r'\w+/\w+:\s(\d+)').astype(float)


    return df

df_clean = clean_games(df)

In [286]:
df_clean

,date,home,away,away_score,home_score,postseason_info,walks_issued_away,walks_issued_home,stolen_bases_away,stolen_bases_home,...,loss_pitcher_er,loss_pitcher_k,loss_pitcher_bb,loss_pitcher_wins,loss_pitcher_losses,save_pitcher_outs,save_pitcher_er,save_pitcher_k,save_pitcher_bb,save_pitcher_saves
game,,,,,,,,,,,,,,,,,,,,,
360403123,2016-04-03,PIT,STL,1.0,4.0,Regular Season,5.0,5.0,0.0,0.0,...,3,3,3,0,1,0,0,0,0,0
360403130,2016-04-03,TB,TOR,5.0,3.0,Regular Season,1.0,3.0,0.0,0.0,...,2,12,3,0,1,3,0,2,0,1
360403107,2016-04-04,KC,NYM,3.0,4.0,Regular Season,2.0,6.0,0.0,1.0,...,3,2,2,0,1,3,0,2,1,1
360404108,2016-04-04,MIL,SF,12.0,3.0,Regular Season,5.0,5.0,1.0,0.0,...,4,2,2,0,1,0,0,0,0,0
360404101,2016-04-04,BAL,MIN,2.0,3.0,Regular Season,5.0,0.0,0.0,1.0,...,1,1,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401361977,2021-10-28,HOU,ATL,2.0,7.0,"World Series - Game 2, ATL Wins series 4-2",2.0,2.0,0.0,2.0,...,6,6,1,0,1,0,0,0,0,0
401361978,2021-10-30,ATL,HOU,0.0,2.0,"World Series - Game 3, ATL Wins series 4-2",5.0,3.0,1.0,0.0,...,1,6,4,0,1,3,0,0,0,1
401361979,2021-10-31,ATL,HOU,2.0,3.0,"World Series - Game 4, ATL Wins series 4-2",3.0,5.0,1.0,0.0,...,2,1,0,0,1,3,0,1,0,2


In [287]:
df_clean.dtypes

date                   object
home                   object
away                   object
away_score            float64
home_score            float64
                       ...   
save_pitcher_outs       int64
save_pitcher_er         int64
save_pitcher_k          int64
save_pitcher_bb         int64
save_pitcher_saves      int64
Length: 61, dtype: object

In [288]:
df_clean.describe()

,away_score,home_score,walks_issued_away,walks_issued_home,stolen_bases_away,stolen_bases_home,strikeouts_thrown_away,strikeouts_thrown_home,total_bases_away,total_bases_home,...,loss_pitcher_er,loss_pitcher_k,loss_pitcher_bb,loss_pitcher_wins,loss_pitcher_losses,save_pitcher_outs,save_pitcher_er,save_pitcher_k,save_pitcher_bb,save_pitcher_saves
count,13295.000000,13295.000000,13295.000000,13295.000000,13295.000000,13295.000000,13295.000000,13295.000000,13295.000000,13295.000000,...,13295.000000,13295.000000,13295.000000,13295.000000,13295.000000,13295.000000,13295.000000,13295.000000,13295.000000,13295.000000
mean,4.524182,4.654607,3.287476,3.184505,0.491087,0.498082,8.159383,8.799248,14.284393,14.133659,...,3.338849,3.075743,1.575329,2.812486,3.701843,1.587364,0.058518,0.626025,0.133358,5.289658
std,3.232759,3.204010,2.045909,2.054259,0.820816,0.833413,2.984556,2.977161,6.763158,6.524857,...,1.983846,2.566004,1.330751,3.141772,2.874645,1.822586,0.276244,0.944949,0.399578,8.900548
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,6.000000,7.000000,9.000000,9.000000,...,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,4.000000,3.000000,3.000000,0.000000,0.000000,8.000000,9.000000,13.000000,13.000000,...,3.000000,3.000000,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.000000,6.000000,4.000000,4.000000,1.000000,1.000000,10.000000,11.000000,18.000000,18.000000,...,5.000000,5.000000,2.000000,4.000000,5.000000,3.000000,0.000000,1.000000,0.000000,7.000000
max,24.000000,29.000000,13.000000,14.000000,7.000000,8.000000,26.000000,26.000000,50.000000,50.000000,...,12.000000,14.000000,8.000000,20.000000,19.000000,17.000000,5.000000,8.000000,5.000000,57.000000


In [289]:
    df_clean.select_dtypes(include=[object]).describe()

,date,home,away,postseason_info,stadium,location,umpires,win_pitcher_name,win_pitcher_abbrname,loss_pitcher_name,loss_pitcher_abbrname,save_pitcher_name,save_pitcher_abbrname,time,fav
count,13295,13295,13295,13295,13295,13295,13295,13295,13295,13295,13295,6603,6603,13295,3211
unique,1108,31,31,307,67,56,8115,1189,1154,1247,1212,559,550,155,31
top,2020-09-26,HOU,HOU,Regular Season,Dodger Stadium,"Chicago, Illinois","Home Plate Umpire - Mike Winters, First Base U...",Max Scherzer,M. Scherzer,Patrick Corbin,A. Sanchez,Kenley Jansen,K. Jansen,23:10:00,LAD
freq,23,474,469,12613,473,881,13,88,88,65,67,221,221,1753,221


In [290]:
df_num = df_clean.select_dtypes(include = [float, int, 'datetime64[ns, UTC]']).filter(regex = r'.+(?<!id)$').drop(['odds','o_u', 'datetime', 'attendance', 'capacity'], axis=1)

In [291]:
df_num.dtypes

away_score                float64
home_score                float64
walks_issued_away         float64
walks_issued_home         float64
stolen_bases_away         float64
stolen_bases_home         float64
strikeouts_thrown_away    float64
strikeouts_thrown_home    float64
total_bases_away          float64
total_bases_home          float64
duration                    int64
away_wins                   int64
away_losses                 int64
awayaway_wins               int64
awayaway_losses             int64
home_wins                   int64
home_losses                 int64
homehome_wins               int64
homehome_losses             int64
win_pitcher_outs            int64
win_pitcher_er              int64
win_pitcher_k               int64
win_pitcher_bb              int64
win_pitcher_wins            int64
win_pitcher_losses          int64
loss_pitcher_outs           int64
loss_pitcher_er             int64
loss_pitcher_k              int64
loss_pitcher_bb             int64
loss_pitcher_w

In [292]:
df_num.isna().sum().sum()
import re
r_home = r'(?<!home)(home)'
r_away = r'(?<!away)(away)'

df_clean.rename(lambda x: re.sub(r_home, r'team', x), axis = 1)


,date,team,away,away_score,team_score,postseason_info,walks_issued_away,walks_issued_team,stolen_bases_away,stolen_bases_team,...,loss_pitcher_er,loss_pitcher_k,loss_pitcher_bb,loss_pitcher_wins,loss_pitcher_losses,save_pitcher_outs,save_pitcher_er,save_pitcher_k,save_pitcher_bb,save_pitcher_saves
game,,,,,,,,,,,,,,,,,,,,,
360403123,2016-04-03,PIT,STL,1.0,4.0,Regular Season,5.0,5.0,0.0,0.0,...,3,3,3,0,1,0,0,0,0,0
360403130,2016-04-03,TB,TOR,5.0,3.0,Regular Season,1.0,3.0,0.0,0.0,...,2,12,3,0,1,3,0,2,0,1
360403107,2016-04-04,KC,NYM,3.0,4.0,Regular Season,2.0,6.0,0.0,1.0,...,3,2,2,0,1,3,0,2,1,1
360404108,2016-04-04,MIL,SF,12.0,3.0,Regular Season,5.0,5.0,1.0,0.0,...,4,2,2,0,1,0,0,0,0,0
360404101,2016-04-04,BAL,MIN,2.0,3.0,Regular Season,5.0,0.0,0.0,1.0,...,1,1,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401361977,2021-10-28,HOU,ATL,2.0,7.0,"World Series - Game 2, ATL Wins series 4-2",2.0,2.0,0.0,2.0,...,6,6,1,0,1,0,0,0,0,0
401361978,2021-10-30,ATL,HOU,0.0,2.0,"World Series - Game 3, ATL Wins series 4-2",5.0,3.0,1.0,0.0,...,1,6,4,0,1,3,0,0,0,1
401361979,2021-10-31,ATL,HOU,2.0,3.0,"World Series - Game 4, ATL Wins series 4-2",3.0,5.0,1.0,0.0,...,2,1,0,0,1,3,0,1,0,2


In [293]:
def get_schedule(df, team):
    df_home =  df[(df.home == team)]
    df_away = df[(df.away == team)]

    df_home['venue'] = df_home.home.apply(lambda x : 'HOME')
    df_away['venue'] = df_away.home.apply(lambda x : 'AWAY')

    r_home = r'(?<!home)(home)'
    r_away = r'(?<!away)(away)'


    df_home = df_home.rename(lambda x: re.sub(r_home, 'team', x), axis = 1).rename(lambda x: re.sub(r_away, 'OPP', x), axis = 1)
    df_away = df_away.rename(lambda x: re.sub(r_away, 'team', x), axis = 1).rename(lambda x: re.sub(r_home, 'OPP', x), axis = 1)
    df =  pd.concat([df_home, df_away]).sort_values('datetime')
    temp = df.filter(regex = r'(wins|losses)$').fillna(method = 'ffill').fillna(0)
    df[temp.columns] = temp
    return df

In [294]:
get_schedule(df_clean, 'BOS')

,date,team,OPP,OPP_score,team_score,postseason_info,walks_issued_OPP,walks_issued_team,stolen_bases_OPP,stolen_bases_team,...,save_pitcher_outs,save_pitcher_er,save_pitcher_k,save_pitcher_bb,save_pitcher_saves,venue,teamaway_wins,teamaway_losses,OPPhome_wins,OPPhome_losses
game,,,,,,,,,,,,,,,,,,,,,
360405105,2016-04-05,BOS,CLE,2.0,6.0,Regular Season,6.0,3.0,0.0,0.0,...,0,0,0,0,0,AWAY,1.0,0.0,0.0,1.0
360406105,2016-04-06,BOS,CLE,7.0,6.0,Regular Season,3.0,6.0,1.0,1.0,...,3,0,1,0,1,AWAY,1.0,1.0,1.0,1.0
360408114,2016-04-08,BOS,TOR,7.0,8.0,Regular Season,3.0,5.0,1.0,0.0,...,3,0,1,0,1,AWAY,2.0,1.0,0.0,1.0
360409114,2016-04-09,BOS,TOR,4.0,8.0,Regular Season,4.0,1.0,1.0,1.0,...,0,0,0,0,0,AWAY,3.0,1.0,0.0,2.0
360410114,2016-04-10,BOS,TOR,3.0,0.0,Regular Season,2.0,4.0,0.0,0.0,...,3,0,3,0,3,AWAY,3.0,2.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401361963,2021-10-16,BOS,HOU,5.0,9.0,"ALCS - GAME 2, HOU Wins series 4-2",4.0,2.0,1.0,0.0,...,0,0,0,0,0,AWAY,43.0,38.0,51.0,30.0
401361966,2021-10-19,BOS,HOU,3.0,12.0,"ALCS - GAME 3, HOU Wins series 4-2",5.0,2.0,0.0,1.0,...,0,0,0,0,0,HOME,43.0,38.0,51.0,30.0
401361967,2021-10-20,BOS,HOU,9.0,2.0,"ALCS - GAME 4, HOU Wins series 4-2",7.0,5.0,0.0,1.0,...,0,0,0,0,0,HOME,43.0,38.0,51.0,30.0


In [300]:
def get_averages(df, team, window = [1,3]):
    sched = get_schedule(df, team).filter(regex = r'(datetime)|(venue)|(.*team.*)').drop('team')

    return sched

In [298]:
get_averages(df_clean, 'BOS')

TypeError: get_averages() missing 1 required positional argument: 'window'